I have a prototype of they system for corrupting sentences, for different semantic meaning:

My process is:

1. Tokenize the sentence (Currently using the NLTK regex tokenizer, it seem sufficient)
2. Parts of Speech tag (Currently using the Stanford POS Tagger (via NLTK))
3. For each word that is not blacklisted (I have blacklisted "had", "were", "have", "was", and "be", as they are unusual verbs with strange antoymns, further more the are rather syntatic)
        1. Use WordNet to find antonyms of the same POS tag  (So "race" (Noun, as in a competition) has no antonyms, but "race" (Verb, to race the train) has "linger" as an antonym.
        2. Unstem: WordNet stemming/lemmaisation (of the antonym) removes Tense, Plurality, comparativeness, and superlativeness, so I make use of the POS tag of the original to work those out, then restore them using the Pattern library's tools for this (http://jmlr.csail.mit.edu/papers/volume13/desmedt12a/desmedt12a.pdf)
        3. I remove any suggested antonyms that are short phrases (eg Wordnet suggests that "take_away" is an antonym of "add", however as adding a work word change the structure of the sentence.)
        4. select one randomly if there are multiple.
4. I incrementally subsitute one addional antynym until I have run out of words with antonys, saving the sentence at each step. (What I'm currentl;y callign each sentence of different corruption level)
5. I repair the indefinite articles ('an' vs 'a')

I can check the final sentence by sending it through the POS tagger and seeing if I get the same tags.
This is not perfect. Its not bad though.
It got a lot better when I changed to using the Stanford POS tagger, as it was more able to tag and retag correctly and thus was most consistent.

The whole method is very heurustic.
It's not too bad but makes a lot of unnatural sounding sentences.
And will sometimes choose the wrong antonym.

For example stating with the sentence:
 - It's not too bad but makes a lot of unnatural sounding sentences.
 - It 's not too unregretful but makes a lot of unnatural sounding sentences.
 - It 's not too unregretful but makes a lot of unnatural devoicing sentences.
 - It 's not too unregretful but makes a lot of unaffected devoicing sentences.
 - It 's not too unregretful but makes a lot of unaffected devoicing acquittals.

In [1]:
from SemanticCorruption import *

In [2]:
import pattern.en as en

In [4]:
sent = "It's not too bad but makes a lot of unnatural sounding sentences."
list(leveled_semantic_corrupt_sentences(sent))

[u"It 's not too bad but makes a lot of unnatural sounding acquittals .",
 u"It 's not too unregretful but makes a lot of unnatural sounding acquittals .",
 u"It 's not too unregretful but makes a lot of unnatural devoicing acquittals .",
 u"It 's not too unregretful but makes a lot of unaffected devoicing acquittals ."]

In [6]:
sent = 'A Washington County man may have the countys first human case of West Nile virus, the health department said Friday'
list(leveled_semantic_corrupt_sentences(sent))

[u'a Washington County man may have the countys first nonhuman case of West Nile virus , the health department said Friday',
 u'an Washington County woman may have the countys first nonhuman case of West Nile virus , the health department said Friday',
 u'an Washington County woman may have the countys first nonhuman case of West Nile virus , the illness department said Friday']

In [2]:
sent = "It was a dusty, dry, hot day, and the flys were buzzing."
list(leveled_semantic_corrupt_sentences(sent))

[u'It was an dusty , dry , hot night , and the flys were buzzing .',
 u'It was an dusty , sweet , hot night , and the flys were buzzing .',
 u'It was an dusty , sweet , cold night , and the flys were buzzing .']

In [3]:
checked_corruption("The article is the most common determiner (DT) in English.")

failed on: (most) RBS  != JJS (least)


u'The article is the least uncommon determiner ( DT ) in English .'

In [4]:
checked_corruption("We may have a question")

u'We may have an answer'

In [5]:
sent="Is changing an odd number of verbs, adverbs, adjectives and nouns to their antonyms expected to produce a semantically distant sentence?"
checked_corruption(sent)


failed on: (expected) VBN  != JJ (unexpected)


u'Is staying an even number of verbs , adverbs , adjectives and nouns to their synonyms unexpected to produce an semantically close acquittal ?'

In [5]:
checked_corruption("Both gerunds and infinitives can be used as the subject or the complement of a sentence.")

u'Both gerunds and infinitives can be misused as the subject or the complement of an acquittal .'

In [6]:
sent = "Shares of Xoma fell 16 percent in early trade, while shares of Genentech, a much larger company with several products on the market, were up 2 percent."
checked_corruption(sent)

u'Shares of Xoma ascended 16 percent in late trade , while shares of Genentech , an much less company with several products on the market , were downwardly 2 percent .'

In [7]:
checked_corruption("Six months ago, the IMF and Argentina struck a bare-minimum $6.8-billion debt rollover deal that expires in August.")

u'Six months ago , the IMF and Argentina missed an bare-minimum $ 6.8-billion debt rollover deal that expires in August .'

In [8]:
checked_corruption("He plans to have dinner with troops at Kosovo's U.S. military headquarters, Camp Bondsteel.")

u"He plans to have dinner with troops at Kosovo 's U.S. civilian headquarters , Camp Bondsteel ."

In [9]:
checked_corruption("After that, he plans to have dinner at Camp Bondsteel with U.S. troops stationed there.")

failed on: (stationed) VBD  != VBN (stationed)


u'After that , he plans to have dinner at Camp Bondsteel with U.S. troops stationed here .'

In [10]:
checked_corruption("He added that prosecutors will seek the death penalty.")

u'He subtracted that prosecutors will seek the birth reward .'

In [11]:
checked_corruption("Who is this man?")

u'Who is this woman ?'

In [12]:
checked_corruption("Who is that man?")

u'Who is that woman ?'

In [13]:
checked_corruption("This evil thief stole that car!")

u'This good thief stole that car !'

In [21]:
checked_corruption("The motorist is angry, so the pedestrian is understandably scared")

u'The motorist is unangry , so the pedestrian is unintelligibly scared'

In [22]:
checked_corruption("The hero was brave, he defeated the dragon.")


'The hero was brave , he defeated the dragon .'

In [23]:
checked_corruption("The villian was cowardly, he was slain by the dragon.")


failed on: (cowardly) JJ  != VB (brave)


u'The villian was brave , he was slain by the dragon .'

In [33]:
checked_corruption("War and Peace opens in the Russian city of St. Petersburg in 1805, as Napoleon's conquest of western Europe is just beginning to stir fears in Russia.")

u"War and Peace opens in the Russian city of St. Petersburg in 1805 , as Napoleon 's conquest of eastern Europe is just ending to stir fearlessnesses in Russia ."

In [15]:
list(get_all_antonyms("race", wn.VERB))

[u'linger']